### Original dictionary
The Wordle dictionary, sorted alphabetically, is stored in wordle_dictionary.py

In [16]:
from wordle_vocab import dictionary

import pandas as pd
import numpy as np
import json


### Create letter scores
The score for each letter is the total number of times it occurs in the dictionary

In [5]:
dictionary.reverse() # so that final dictionary will be sorted alphabetically within equal scores

str_dictionary = ''.join(dictionary)
alphabet = 'abcdefghijklmnopqrstuvwxyz'
alphabet_list = [a for a in alphabet]
letter_scores_list = [str_dictionary.count(letter) for letter in alphabet]

# Score of each letter is just the number of times it appears in the dictionary
letter_scores = dict(zip(alphabet_list,letter_scores_list))

### Here are the letter scores this generates

In [4]:
letter_scores = {'a': 5990, 'b': 1627, 'c': 2028, 'd': 2453, 'e': 6662, 'f': 1115, 'g': 1644, 'h': 1760, 'i': 3759, 'j': 291, 'k': 1505, 'l': 3371, 'm': 1976, 'n': 2952, 'o': 4438, 'p': 2019, 'q': 112, 'r': 4158, 's': 6665, 't': 3295, 'u': 2511, 'v': 694, 'w': 1039, 'x': 288, 'y': 2074, 'z': 434}

### Create dictionary scores
For each word in the dictionary, the score is the sum of its *unique* letter scores

In [7]:
# Score of a word is the sum of the *unique* letter scores in each word
dictionary_scores_list = [sum([letter_scores[letter] for letter in set(word)]) for word in dictionary]
dictionary_scores = dict(zip(dictionary,dictionary_scores_list))
scored_dictionary = sorted(dictionary_scores, key=dictionary_scores.get)
scored_dictionary.reverse() # Sort by score *descending*

In [8]:
scored_dictionary

['soare',
 'arose',
 'aeros',
 'serai',
 'reais',
 'raise',
 'arise',
 'aesir',
 'aloes',
 'toeas',
 'stoae',
 'seral',
 'reals',
 'rales',
 'lears',
 'laser',
 'lares',
 'laers',
 'earls',
 'arles',
 'teras',
 'tears',
 'taser',
 'tares',
 'strae',
 'stear',
 'stare',
 'resat',
 'reast',
 'rates',
 'earst',
 'aster',
 'arets',
 'aeons',
 'aisle',
 'snare',
 'saner',
 'reans',
 'nears',
 'nares',
 'earns',
 'saine',
 'isnae',
 'anise',
 'ursae',
 'ureas',
 'urase',
 'aures',
 'tesla',
 'teals',
 'tales',
 'taels',
 'stela',
 'steal',
 'stale',
 'slate',
 'setal',
 'salet',
 'leats',
 'least',
 'sared',
 'reads',
 'rased',
 'eards',
 'dears',
 'dares',
 'arsed',
 'psoae',
 'paseo',
 'osier',
 'slane',
 'neals',
 'leans',
 'lanes',
 'elans',
 'stean',
 'stane',
 'neats',
 'nates',
 'etnas',
 'antes',
 'years',
 'sayer',
 'resay',
 'eyras',
 'ayres',
 'arsey',
 'ideas',
 'aside',
 'aides',
 'serac',
 'scrae',
 'scare',
 'races',
 'escar',
 'carse',
 'cares',
 'acres',
 'acers',
 'spear',


This final dictionary, sorted by word score, is included in ```WordleBot/wordle_dictionary.py```

## Create Popularity Scores

You need to first run `scrape_word_popularities.py` to obtain a csv file containing all of the popularity scores for every word in Wordle's vocabulary from Google Books. We then rework that into a nice dictionary of scores that can be used by Wordle bot.

Current plan is to take the average popularity score of each word (case insensitive) from the past ten years. That will be a small offset we add to the entropy score of each word in entropy_guesser.

In [2]:
df=pd.read_csv("word_popularity_data.csv.zip",compression="zip")


In [4]:
def score_averager(x,n_years=10):
    return np.mean(x[-n_years:])

In [7]:
df["average_popularity"]=df.apply(score_averager,axis=1)

In [9]:
idx=df["case"]=="Insensitive"
columns=["word","case","average_popularity"]
df.loc[idx,columns]

,word,case,average_popularity
0,aahed,Insensitive,4.629283e-08
1,aalii,Insensitive,4.632601e-10
3,aargh,Insensitive,4.016478e-08
5,aarti,Insensitive,8.236386e-08
7,abaca,Insensitive,4.356348e-08
...,...,...,...
25738,zuzim,Insensitive,7.567169e-09
25740,zygal,Insensitive,9.703162e-10
25742,zygon,Insensitive,4.902521e-08
25744,zymes,Insensitive,8.497195e-09


In [21]:
df.loc[idx,"word"].str.lower()

0        aahed
1        aalii
3        aargh
5        aarti
7        abaca
         ...  
25738    zuzim
25740    zygal
25742    zygon
25744    zymes
25746    zymic
Name: word, Length: 13038, dtype: object

In [23]:
pop_dict=dict(zip(
    df.loc[idx,"word"].str.lower().values,
    df.loc[idx,"average_popularity"].values
)
)

In [24]:
# Serialize data into file:
json.dump( pop_dict, open( "popularity_dict.json", 'w' ) )

The contents of the above JSON file can just be copy and pasted into WordleBot/wordle_dictionary.py as `popularity_dict`